In [16]:
import pandas as pd
import numpy as np
import requests
import json
from datetime import datetime,timedelta,date
import sys



import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from matplotlib import cm
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from matplotlib.dates import (YEARLY, MONTHLY, DateFormatter, WeekdayLocator, MonthLocator,DayLocator,
                              rrulewrapper, RRuleLocator, drange, num2date, date2num)
import matplotlib.patches as mpatches
import matplotlib.units as munits
from matplotlib.dates import num2date, date2num

import seaborn as sns


import html


from api_keys import token_esios #importo mi token de esios, añadan su propia clave en su versión

In [17]:
token_esios

'tu_token_de_api_aqui'

# Funciones para bajar los datos

In [18]:
def catalogo_esios(token):
    """
    Descarga todos los identificadores y su descripcion de esios
    
    Parameters
    ----------
    token : str
        El token de esios necesario para realizar las llamadas al API
        
    Returns
    -------
    DataFrame
        Dataframe de pandas con el catalogo de los id de la API
    
    """
    
    
    headers = {'Accept':'application/json; application/vnd.esios-api-v2+json',
           'Content-Type':'application/json',
           'Host':'api.esios.ree.es',
           'Cookie' : '',
           'Authorization':'Token token={}'.format(token),
           'Cache-Control': 'no-cache',
           'Pragma': 'no-cache'
          }
    end_point = 'https://api.esios.ree.es/indicators'
    response = requests.get(end_point, headers=headers).json()
    
    #del resultado en json bruto se convierte en pandas, y se eliminan los tags del campo description

    return (pd
            .json_normalize(data=response['indicators'], errors='ignore')
            .assign(description = lambda df_: df_.apply(lambda df__: html.unescape(df__['description']
                                                            .replace('<p>','')
                                                            .replace('</p>','')
                                                            .replace('<b>','')
                                                            .replace('</b>','')), 
                                                  axis=1)
                   )
           )

In [19]:
def download_esios(token,indicadores,fecha_inicio,fecha_fin,time_trunc='day'):
    """
    Descarga datos esios desde un determinado identidficador y entre dos fechas
    
    Parameters
    ----------
    token : str
        El token de esios necesario para realizar las llamadas al API
    
    indicadores : list
        Lista con los strings de los indicadores de los que queremos bajar datos
        
    fecha_inicio : str
        Fecha con formato %Y-%M-%d, que indica la fecha desde la que se quiere bajar los datos.
        Ejemplo 2022-10-30, 30 Octubre de 2022.
    
    fecha_fin : str
        Fecha con formato %Y-%M-%d, que indica la fecha hasta la que se quiere bajar los datos.
        Ejemplo 2022-10-30, 30 Octubre de 2022.
        
    time_trunc : str, optional
        Campo adicional que nos permite elegir la granularidad de los datos que queremos bajar.
        
    Returns
    -------
    DataFrame
        Dataframe de pandas con los datos solicitados
    
    """
    
    # preparamos la cabecera a insertar en la llamada. Vease la necesidad de disponer el token de esios
    
    headers = {'Accept':'application/json; application/vnd.esios-api-v2+json',
           'Content-Type':'application/json',
           'Host':'api.esios.ree.es',
           'Cookie' : '',
           'Authorization':'Token token={}'.format(token),
           'Cache-Control': 'no-cache',
           'Pragma': 'no-cache'
          }
    
    # preparamos la url básica a la que se le añadiran los campos necesarios 
    
    end_point = 'https://api.esios.ree.es/indicators'
    
    # El procedimiento es sencillo: 
    # a) por cada uno de los indicadores configuraremos la url, según las indicaciones de la documentación.
    # b) Hacemos la llamada y recogemos los datos en formato json.
    # c) Añadimos la información a una lista
    
    lista=[]

    for indicador in indicadores:
        url = f'{end_point}/{indicador}?start_date={fecha_inicio}T00:00&\
        end_date={fecha_fin}T23:59&time_trunc={time_trunc}'
        print (url)
        response = requests.get(url, headers=headers).json()
        lista.append(pd.json_normalize(data=response['indicator'], record_path=['values'], meta=['name','short_name'], errors='ignore'))

    # Devolvemos como salida de la función un df fruto de la concatenación de los elemenos de la lista
    # Este procedimiento, con una sola concatenación al final, es mucho más eficiente que hacer múltiples 
    # concatenaciones.
    
    return pd.concat(lista, ignore_index=True )

In [20]:
def download_ree(indicador,fecha_inicio,fecha_fin,time_trunc='day'):
    """
    Descarga datos desde apidatos.ree.es entre dos fechas determinadas 
    
    Parameters
    ----------
    
    indicador : str
        Texto con el indicador del end point del que queremo bajar la información
        
    fecha_inicio : str
        Fecha con formato %Y-%M-%d, que indica la fecha desde la que se quiere bajar los datos.
        Ejemplo 2022-10-30, 30 Octubre de 2022.
    
    fecha_fin : str
        Fecha con formato %Y-%M-%d, que indica la fecha hasta la que se quiere bajar los datos.
        Ejemplo 2022-10-30, 30 Octubre de 2022.
        
    time_trunc : str, optional
        Campo adicional que nos permite elegir la granularidad de los datos que queremos bajar.
        Hour, Day, Month...dependiendo del end point se aplicará o no esta orden
        
    Returns
    -------
    DataFrame
        Dataframe de pandas con los datos solicitados
    
    """
    
    
    headers = {'Accept': 'application/json',
               'Content-Type': 'applic<ation/json',
               'Host': 'apidatos.ree.es'}
    
    end_point = 'https://apidatos.ree.es/es/datos/'
    
    lista=[]
    url = f'{end_point}{indicador}?start_date={fecha_inicio}T00:00&end_date={fecha_fin}T23:59&\
    time_trunc={time_trunc}'
    print (url)
    
    response = requests.get(url, headers=headers).json()
    
    return pd.json_normalize(data=response['included'], 
                                   record_path=['attributes','values'], 
                                   meta=['type',['attributes','type' ]], 
                                   errors='ignore')

In [21]:
def download_gas(year):
    """
    Descarga datos de precio de gas desde MIBGAS para GDAES
    
    Parameters
    ----------
    year : str
        Indicamos el año del que nos queremos bajar los datos de precio de gas PVB
        
    Returns
    -------
    DataFrame
        Dataframe de pandas con los datos solicitados, columnas Fecha , Producto y Precio
    
    """
    
    path = f'https://www.mibgas.es/en/file-access/MIBGAS_Data_{year}.xlsx?path=AGNO_{year}/XLS'
    return (pd.read_excel(path,sheet_name='Trading Data PVB&VTP',usecols=['Trading day','Product','Daily Reference Price\n[EUR/MWh]']).
       query("Product=='GDAES_D+1'").
       rename(columns={'Trading day':'fecha','Product':'Producto','Daily Reference Price\n[EUR/MWh]':'precio'}).
       sort_values('fecha',ascending=True).
       reset_index(drop=True)
      )

In [22]:
def download_gas_rd(year):
    """
    Descarga datos de precio de gas desde MIBGAS para compensación segñun RD10/2022
    
    Parameters
    ----------
    YEAR : str
        Indicamos el año del que nos queremos bajar los datos de precio de gas de RD10/22
        
    Returns
    -------
    DataFrame
        Dataframe de pandas con los datos solicitados, columnas Fecha , Producto y Precio
    
    """
    
    path = f'https://www.mibgas.es/en/file-access/MIBGAS_Data_{year}.xlsx?path=AGNO_{year}/XLS'
    return (pd.read_excel(path,sheet_name='PGN_RD_10_2022',
                          usecols=['Date','PGN Price\n[EUR/MWh]']).
       rename(columns={'Date':'fecha','PGN Price\n[EUR/MWh]':'precio'}).
       sort_values('fecha',ascending=True).
       reset_index(drop=True)
      )

      

# Datos desde https://www.ree.es/es/apidatos

En el link del título tenemos la descripción del servicio `REST` para acceder a la API. Siguiendo esas instrucciones montamos la funcion `download_ree` para que de manera fácil podamos bajar la información que disponemos.

Veamos un ejemplo práctico bajando la generación, con sus diferentes tipos:

In [23]:
fin = datetime.today().strftime('%Y-%m-%d')  # string con la fecha de hoy en el formato requerido por funcion
inicio = (datetime.today()-timedelta(days=7)).strftime('%Y-%m-%d')
identificador = 'generacion/estructura-generacion'

In [30]:
raw = download_ree(identificador,inicio,fin)
raw

https://apidatos.ree.es/es/datos/generacion/estructura-generacion?start_date=2023-11-05T00:00&end_date=2023-11-12T23:59&    time_trunc=day


,value,percentage,datetime,type,attributes.type
0,88871.500,0.139916,2023-11-05T00:00:00.000+01:00,Hidráulica,Renovable
1,111999.100,0.157790,2023-11-06T00:00:00.000+01:00,Hidráulica,Renovable
2,125630.800,0.178193,2023-11-07T00:00:00.000+01:00,Hidráulica,Renovable
3,125013.900,0.180512,2023-11-08T00:00:00.000+01:00,Hidráulica,Renovable
4,129460.800,0.171445,2023-11-09T00:00:00.000+01:00,Hidráulica,Renovable
...,...,...,...,...,...
97,709797.602,1.000000,2023-11-06T00:00:00.000+01:00,Generación total,Generación total
98,705027.478,1.000000,2023-11-07T00:00:00.000+01:00,Generación total,Generación total
99,692552.066,1.000000,2023-11-08T00:00:00.000+01:00,Generación total,Generación total
100,755114.387,1.000000,2023-11-09T00:00:00.000+01:00,Generación total,Generación total


Veamos un ejemplo de los datos descargados:

Tal como hicimos en el caso anterior vamos a formatear los datos en bruto para sacar la tabla que queremos:

In [25]:
raw.dtypes

value              float64
percentage         float64
datetime            object
type                object
attributes.type     object
dtype: object

In [33]:
generacion = (raw
              .assign(fecha=lambda df_: pd
                      .to_datetime(df_['datetime'],utc=True)
                      .dt
                      .tz_convert('Europe/Madrid')
                      .dt
                      .tz_localize(None)
                      )
              .drop(['attributes.type','datetime'],axis=1)
              .rename(columns={'value':'valor','type':'tipo','value':'generacion'})[['fecha','tipo','generacion','percentage']]
            )

y nos queda tal que así

In [34]:
generacion

,fecha,tipo,generacion,percentage
0,2023-11-05,Hidráulica,88871.500,0.139916
1,2023-11-06,Hidráulica,111999.100,0.157790
2,2023-11-07,Hidráulica,125630.800,0.178193
3,2023-11-08,Hidráulica,125013.900,0.180512
4,2023-11-09,Hidráulica,129460.800,0.171445
5,2023-11-10,Hidráulica,129801.300,0.165576
12,2023-11-05,Nuclear,78960.800,0.124313
13,2023-11-06,Nuclear,107745.200,0.151797
14,2023-11-07,Nuclear,120433.400,0.170821
15,2023-11-08,Nuclear,120624.400,0.174174


..y los utilizamos para generar un gráfico de barras que nos muestras la evolución de la generación, por cada fuente, en estos días:

In [43]:
import plotly.express as px
import pandas as pd

# Suponiendo que 'generacion' es tu DataFrame
# generacion = pd.read_csv('tu_archivo.csv')

fig = px.bar(
    generacion,
    x='fecha',
    y='percentage',
    color='tipo',
    labels={'percentage': 'Porcentaje'},
    title='Evolución de diferentes tecnologías de generación',
    template='plotly_white',
)


fig.show()
